# **※ 목표**
⚡ 신용거래 이상탐지 데이터를 다뤄보며 Feature Engineering을 바탕으로 나만의 Feature를 만들어 볼 것!
### **1. 실습 미션**
    ◆ 사기거래 여부를 예측하는 데 불필요한 컬럼을 제거하기
    ◇ 최소 2개 이상의 새로운 피처(변수)를 생성해보기
        ▷ '구매 금액'과 '시간' 등의 변수를 이용할 수도 있다!
### **2. 추가 미션**
    ◆ 데이터의 위도/경도 정보도 활용해보기
    ◇ 통계적 관점으로 접근해 유용한 변수를 만들 수 있을까?

### **시작하기 전 ⏰** : 결측치 확인하고 가기

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import pandas as pd
fraud = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fraud.csv')
pd.set_option('display.max_columns', 30) # head()를 바로 진행하면 ...로 칼럼이 감춰지기 때문에 풀어주는 코드부터 작성

In [65]:
fraud.isna().sum() # 이번 데이터는 결측치가 없었다.

,0
trans_date_trans_time,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0
city,0


In [66]:
fraud.info()
# 이 데이터는 약 49만개의 많은 분량 데이터가 있네요..

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [67]:
# 사기 거래는 전체의 몇 % 였을까?

fraud['is_fraud'].value_counts(normalize=True)

# 정상 거래는 99.7%, 사기 거래는 0.03%의 아주 극심한 불균형한 데이터로 확인된다.

,proportion
is_fraud,
0,0.997467
1,0.002533


## **1 - ◆ 사기거래 여부에 불필요한 컬럼은 무엇일까? 😲**
📖 금융(리스크/사기 탐지) 분야에 대한 이해가 부족해 우선 컬럼명을 한글로 변경하며 이해도를 높이는 작업부터 진행

📖 반대로 사기거래 여부에 꼭 필요한 컬럼부터 체크해보고 불필요한 컬럼은 무엇인지 좁혀나가는 작업을 진행

In [68]:
fraud.head(1) # 컬럼명과 데이터 확인을 위해 1개의 행만 열어보았다.

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0


In [69]:
# 컬럼명을 한글로 변경하기
old_cols = fraud.columns
new_cols = ['거래일시','카드번호','가맹점명','가맹점_카테고리','거래금액','이름','성','성별',
            '거리주소','도시','주','우편번호','위도','경도','도시인구','직업',
            '생년월일','거래번호','유닉스시간','가맹점_위도','가맹점_경도','사기여부']

fraud = fraud.rename(columns=dict(zip(old_cols, new_cols)))
fraud.head()

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,이름,성,성별,거리주소,도시,주,우편번호,위도,경도,도시인구,직업,생년월일,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


사기거래 여부에 필수인 컬럼은 아래와 같이 생각했다.

[거래일시, 카드번호, 가맹점명, 가맹점_카테고리, 거래금액, 유닉스시간, 가맹점_위도, 가맹점_경도, 거래번호, 사기여부]

→ 대부분 거래 당사자임을 알 수 있고 개인의 고유 신상정보가 아닌 **행동 패턴에 대한 식별 정보**들이었다.

그 외 컬럼은 [이름, 성, 성별, 거리주소, 도시, 주, 우편번호, 위도, 경도, 도시인구, 직업, 생년월일] 으로 바로 삭제해보기로 했다.

In [70]:
# 12가지 컬럼 삭제
fraud.drop(['이름', '성', '성별', '거리주소', '도시', '주', '우편번호', '위도', '경도', '도시인구', '직업', '생년월일'], axis=1, inplace=True)
fraud.head()

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


## **1 - ◇ 최소 2개 이상의 새로운 피처(변수)를 생성해보기 ✌**

⭐ '구매 금액(거래금액)'을 활용해 '가맹점 카테고리별 평균 대비 현재 거래 금액'을 확인

    ▷ 가맹점 카테고리(그룹화) 평균 금액을 현재 구매자의 금액과 대비해 비정상적인 금액이 거래되진 않았는지 확인

⭐ '시간'을 활용해 '이전 거래 시점과의 시간 간격'을 확인

    ▷ 짧은 시간 내 연속적으로 구매했다면 사기에 해당될 수 있음

In [71]:
# 가맹점 카테고리는 어떤 것들이 있는지 확인

fraud['가맹점_카테고리'].unique()

array(['grocery_pos', 'grocery_net', 'misc_pos', 'gas_transport',
       'misc_net', 'health_fitness', 'travel', 'personal_care',
       'shopping_net', 'shopping_pos', 'home', 'entertainment',
       'food_dining', 'kids_pets'], dtype=object)

In [72]:
#  가맹점 카테고리 평균 대비 거래 금액 피처 생성

fraud['카테고리_평균비'] = fraud.groupby('가맹점_카테고리')['거래금액'].transform('mean').round(1) # 소수점 1자리만
fraud['평균대비_n배'] = (fraud['거래금액']/fraud['카테고리_평균비']).round(1) # 소수점 1자리만
fraud.head()

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부,카테고리_평균비,평균대비_n배
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,126.8,0.8
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0,126.8,0.4
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0,51.4,0.9
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0,126.8,0.5
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0,72.4,0.4


In [73]:
# 각 카테고리 별 평균 금액과 최대 거래 금액을 확인

fraud.groupby('가맹점_카테고리')['거래금액'].agg(['mean', 'max'])

,mean,max
가맹점_카테고리,,
entertainment,73.130371,792.09
food_dining,41.933809,663.92
gas_transport,63.071509,154.03
grocery_net,51.420363,185.81
grocery_pos,126.759293,392.00
health_fitness,54.520994,594.17
home,55.579635,560.81
kids_pets,55.039312,472.76
misc_net,82.121960,4084.34


In [74]:
# 카테고리별 최대 금액의 원본 행을 확인 (gpt 도움을 받음...)
  # idxmax라는 메서드를 쓰면 특정 컬럼에서 가장 큰 값을 가진 행의 인덱스를 반환한다고 함

idx = fraud.groupby('가맹점_카테고리')['거래금액'].idxmax()
fraud.loc[idx]

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부,카테고리_평균비,평균대비_n배
469675,2020-12-14 12:05:03,503874407318,fraud_Howe PLC,entertainment,792.09,9a1dc75a685a09845c599a99d26d8730,1387022703,28.842292,-99.049989,0,73.1,10.8
275696,2020-03-08 12:53:58,4745996322265,fraud_Kilback LLC,food_dining,663.92,4d1cfa0cffac1555bfd56d9ed69b63c6,1362747238,42.404307,-86.169511,0,41.9,15.8
96348,2019-06-18 11:47:51,30199621383748,fraud_Kling Inc,gas_transport,154.03,b73d8c6d9a69f8bdf142f13de17142bc,1340020071,32.928410,-95.572997,0,63.1,2.4
71988,2019-05-15 02:15:56,30551643947183,fraud_Roob LLC,grocery_net,185.81,0f6d00bfc6fdaa1d2607eb958dc1d45e,1337048156,35.660060,-81.414384,0,51.4,3.6
465796,2020-12-11 05:16:27,3596217206093829,fraud_Stracke-Lemke,grocery_pos,392.00,0dda73ba1f5adbd7b92c0b7f1f0b0c2a,1386738987,40.742460,-92.531863,1,126.8,3.1
349157,2020-06-28 20:44:07,4797297220948468262,fraud_Fritsch and Sons,health_fitness,594.17,4b476d7c88411fdb3e299b1950a26513,1372452247,39.842485,-86.710893,0,54.5,10.9
11746,2019-01-27 16:13:21,3576021480694169,fraud_Beier and Sons,home,560.81,b1e06a753aff3b381fee6633872f0005,1327680801,38.483241,-95.254202,0,55.6,10.1
67696,2019-05-07 19:54:46,377026671291680,fraud_Rau-Grant,kids_pets,472.76,e6422cc34787df7f7b94fb9552b6fa5c,1336420486,36.462256,-91.062873,0,55.0,8.6
58091,2019-04-21 21:37:02,374930071163758,fraud_McGlynn-Heathcote,misc_net,4084.34,6bf0a972ce3d1d7a0526180808a4807e,1335044222,43.018872,-83.130319,0,82.1,49.7
341226,2020-06-17 22:46:05,4642255475285942,fraud_McDermott-Rice,misc_pos,2703.62,181e3ec451fb422b808b5646f7726b99,1371509165,38.864848,-109.515060,0,72.4,37.3


가맹점 카테고리가 총 14개였고, 그 중 이미 사기였던 1건을 제외한 13건은 정상 거래였던 것으로 확인했다.

**이는 단순히 거래 금액이 높았다고 해서 사기 거래는 아니라는 것** ❗

그럼 사기 거래가 많이 나타나는 가맹점 카테고리는 뭘까?

In [75]:
가맹점별_사기거래_비율 = fraud.groupby('가맹점_카테고리')['사기여부'].mean().sort_values(ascending=False)
가맹점별_사기거래_비율

,사기여부
가맹점_카테고리,
shopping_net,0.006177
grocery_pos,0.005056
misc_net,0.004329
shopping_pos,0.002772
gas_transport,0.002194
travel,0.001902
misc_pos,0.001759
food_dining,0.001723
kids_pets,0.001688


인터넷 쇼핑과 식료품 대면 결제, 기타 대면 결제에서 사기 여부가 가장 많았던 것으로 확인했다.

우리나라도 인터넷 쇼핑에서 사기 거래(구매까지 이뤄진 후 사이트가 없어진다던지...)가 많기 때문에 충분히 예상되는 수치라고 볼 수 있었다.

다음으로는 '시간'을 활용해 이전에 거래한 데이터 중 사기 여부는 어떤지 확인했다.

In [76]:
# '거래일시'을 활용해 이전에 거래한 데이터를 추출

fraud['이전거래일시'] = fraud.groupby('카드번호')['거래일시'].shift()
notna = fraud[fraud['이전거래일시'].notna()]
notna

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부,카테고리_평균비,평균대비_n배,이전거래일시
10,2019-01-01 00:41:45,4642255475285942,fraud_Huel-Langworth,misc_net,177.57,048dc7fc7ae8efd51f59f7176e6a8bd7,1325378505,38.419431,-109.380252,0,82.1,2.2,2019-01-01 00:31:51
20,2019-01-01 01:09:57,180048185037117,"fraud_Rippin, Kub and Mann",misc_net,3.79,7c63dd07b14aac7dc99f4587d83dd599,1325380197,40.813669,-75.283465,0,82.1,0.0,2019-01-01 00:17:16
45,2019-01-01 03:10:00,4587657402165341815,fraud_Friesen-D'Amore,gas_transport,77.59,24bc3a291efe739d2f801fdc487ee3f2,1325387400,41.818260,-88.129609,0,63.1,1.2,2019-01-01 03:04:28
47,2019-01-01 03:47:28,3583635130604947,fraud_Bauch-Raynor,grocery_pos,205.79,2fdd291bd3d6fc707973f3e920d9e815,1325389648,40.650005,-74.298184,0,126.8,1.6,2019-01-01 01:49:37
50,2019-01-01 03:49:56,4646845581490336108,fraud_Block Group,misc_pos,1.10,6b10287ffb26869d974ecf9db719f630,1325389796,40.478410,-72.974367,0,72.4,0.0,2019-01-01 01:59:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,fraud_Botsford and Sons,home,134.26,fc860b0d1f89b0b068c9c8db27b6bcc5,1388534208,34.853497,-78.664158,0,55.6,2.4,2020-12-31 23:30:29
491130,2020-12-31 23:56:57,213112402583773,"fraud_Baumbach, Hodkiewicz and Walsh",shopping_pos,25.49,0f0c38fe781b317f733b845c0d6ba448,1388534217,35.008839,-81.475156,0,82.9,0.3,2020-12-31 22:29:20
491131,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0,55.0,2.0,2020-12-31 21:16:46
491132,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0,55.0,1.6,2020-12-31 23:47:11


카드 번호를 그룹화한 뒤 첫 결제 이후 추가 거래 이력이 있는지 확인했다.

그리고 이전 거래 이력이 없는 NaT 데이터는 일단 제외했다.

In [77]:
# 이전 거래가 있는 행들만 대상으로 사기/정상 비율 확인
fraud[fraud['이전거래일시'].notna()]['사기여부'].value_counts(normalize=True)

,proportion
사기여부,
0,0.997466
1,0.002534


(전체 사기 여부 중 정상(0.997467%), 사기(0.002533%))

아무래도 사기 거래의 수치가 매우 적기 때문에 단순 수치화를 하면 영향이 거의 미치지 않는 것으로 보인다.

그럼 이전 거래 이력과 사기 여부의 비례는 없는 걸까?

In [78]:
has_log  = fraud[fraud['이전거래일시'].notna()]
no_log   = fraud[fraud['이전거래일시'].isna()]
print("이전 거래 있을 때 사기 비율:", has_log['사기여부'].mean())
print("이전 거래 없을 때 사기 비율:", no_log['사기여부'].mean())

이전 거래 있을 때 사기 비율: 0.0025335532881204047
이전 거래 없을 때 사기 비율: 0.0


이전 거래 이력이 없으면 사기는 발생하지 않았고 이전 거래 이력이 있는 경우에만 사기가 발생했다.

즉 **사기 거래는 이미 사용된 카드에서 발생한다는 것**이었다.

이러한 피처들을 모두 포함해 사기 패턴을 분석하고 모델 성능 개선을 진행하려고 한다.

이때 문자열은 머신러닝 알고리즘 시 숫자형으로 변경할 필요가 있기 때문에 사전 작업부터 진행하겠다!

In [79]:
# 문자 컬럼 찾기

fraud.describe(include='object')

,거래일시,가맹점명,가맹점_카테고리,거래번호,이전거래일시
count,491134,491134,491134,491134,491010
unique,488474,693,14,491134,488351
top,2020-04-07 11:34:26,fraud_Kilback LLC,shopping_pos,1765bb45b3aa3224b4cdcb6e7a96cee3,2020-11-16 13:51:37
freq,3,1602,48345,1,3


이 데이터에서 문자 형식은 거래일시, 가맹점명, 가맹점_카테고리, 거래번호였다.

모든 문자열을 숫자형식으로 변환하는 것도 좋지만, ***사실상 사용하는 컬럼에 대해서만 숫자형으로 변환***하고 머신러닝에 적용하려고 한다.

거래일시는 datetime 형식으로 변환하고 가장 많이 사용하는 가맹점_카테고리를 숫자형으로 변환하고자 한다.

나는 이미 가맹점_카테고리의 리스트를 알고 있으니 순서대로 1번~14번으로 지정한 뒤 규칙적인 숫자로 지정하려고 한다.


[가맹점_카테고리 넘버링]
1. 'grocery_pos'
2. 'grocery_net'
3. 'misc_pos'
4. 'gas_transport'
5. 'misc_net'
6. 'health_fitness'
7. 'travel'
8. 'personal_care'
9. 'shopping_net'
10. 'shopping_pos'
11. 'home'
12. 'entertainment'
13. 'food_dining'
14. 'kids_pets'



In [80]:
fraud['거래일시'] = pd.to_datetime(fraud['거래일시']) # datetime으로 변환
categories = ['grocery_pos', 'grocery_net', 'misc_pos', 'gas_transport',
              'misc_net', 'health_fitness', 'travel', 'personal_care',
              'shopping_net', 'shopping_pos', 'home', 'entertainment',
              'food_dining', 'kids_pets']
mapping = {cat: i+1 for i, cat in enumerate(categories)}

fraud['카테고리_번호'] = fraud['가맹점_카테고리'].map(mapping).astype(int)

In [81]:
from sklearn.model_selection import train_test_split

# 1) X, y 분리
X = fraud.select_dtypes(include=['int64', 'float64']).drop(columns=['사기여부']) # 숫자형만 선택해서 머신러닝 적용
y = fraud['사기여부']

# 2) train / test 분리 (비율 7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from sklearn.preprocessing import StandardScaler # 스케일링
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 -> 분류 모델
from sklearn.metrics import confusion_matrix, classification_report

smote = SMOTE(random_state=42)

#gpt의 도움을 매우 많이 받음... 로지스틱 모델을 통해 알고리즘 생성, 파이프라인을 작동해 자동화 작업까지 진행
log_clf = LogisticRegression(class_weight={0:1, 1:1},
                             max_iter=1000,
                             n_jobs=-1)
pipe = ImbPipeline(steps=[('scaler', StandardScaler()),
                          ('smote', smote),
                          ('clf', log_clf)])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print("Confusion matrix")
print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred, digits=3))

Confusion matrix
[[140560   6408]
 [   140    233]]
              precision    recall  f1-score   support

           0      0.999     0.956     0.977    146968
           1      0.035     0.625     0.066       373

    accuracy                          0.956    147341
   macro avg      0.517     0.791     0.522    147341
weighted avg      0.997     0.956     0.975    147341



혼동 행렬을 해석하자면, 전체 데이터 49만건 중 30%를 테스트 데이터로 활용해 약 15만건에 대한 값을 예측했고 아래와 같이 정상 및 사기 판단을 진행했다.

수치도 해석해보면,

- TN (140,560건) : 옳게 판단한 Negative, 정상 거래를 정상 거래라고 알맞게 판단한 경우
- FP (6,408건) : 틀리게 판단한 Positive, 정상 거래를 사기 거래라고 잘못 판단한 경우
- FN (140건) : 틀리게 판단한 Negative, 사기 거래를 정상 거래라고 잘못 판단한 경우
- TP (233건) : 옳게 판단한 Positive, 사기 거래를 사기 거래라고 알맞게 판단한 경우

이 머신러닝에서 정확도를 구하자면,

    TP(233)+TN(140,560) = 140,793
    FP(6,408)+FN(140) = 6,548
    140,793 / 147,341 = 0.95

로 높은 정확도를 가졌지만 사기 탐지를 예측하는 머신러닝에선 정확도가 낮고 recall이 0.8~0.9정도 되어야 더 완벽한 머신러닝이라고 한다.

그래서 나는 이 데이터에서 사기를 놓치는 것에 대한 실제 금전적 손실 발생에 중점을 두고 싶어서 recall을 높이는 것을 목표로 삼았다.

현재 사기의 37.5%를 놓치고 있어서 가중치를 높여서 recall의 값을 맞춰보고자 한다.

In [82]:
log_clf = LogisticRegression(class_weight={0:1, 1:2},
                             max_iter=1000,
                             n_jobs=-1)
pipe = ImbPipeline(steps=[('scaler', StandardScaler()),
                          ('smote', smote),
                          ('clf', log_clf)])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print("Confusion matrix")
print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred, digits=3))

Confusion matrix
[[91570 55398]
 [   84   289]]
              precision    recall  f1-score   support

           0      0.999     0.623     0.767    146968
           1      0.005     0.775     0.010       373

    accuracy                          0.623    147341
   macro avg      0.502     0.699     0.389    147341
weighted avg      0.997     0.623     0.766    147341



사기에 대한 가중치를 2배 높여보았을 때

- TN (91,570건) : 옳게 판단한 Negative, 정상 거래를 정상 거래라고 알맞게 판단한 경우
- FP (55,398건) : 틀리게 판단한 Positive, 정상 거래를 사기 거래라고 잘못 판단한 경우
- FN (84건) : 틀리게 판단한 Negative, 사기 거래를 정상 거래라고 잘못 판단한 경우
- TP (289건) : 옳게 판단한 Positive, 사기 거래를 사기 거래라고 알맞게 판단한 경우

      TP(289)+TN(91,570) = 91,859
      FP(55,398)+FN(84) = 55,482
      91,859 / 147,341 = 0.62

사기 거래를 잡아내는 것은 조금 나아졌지만, 정상 거래에 대한 오탐이 너무 늘어나서 오히려 정확도가 떨어지는 것을 확인할 수 있었다.

사기 거래를 더 많이 잡기 위해 무작정 임계값을 높게 잡는 것은 오히려 정상 거래를 사기 거래로 잘못 탐지할 수 있기 때문에 밸런스를 잡는 게 가장 중요할 것 같다..

## **개인정보보호 및 보안**

개인 정보가 들어간 데이터는 모두 익명화를 필수로 진행해야되어 문자열로 치환하는 방식으로 진행

In [90]:
fraud['카드번호'] = "[CARD]"
fraud['거래번호'] = "[TRANS]"
fraud

,거래일시,카드번호,가맹점명,가맹점_카테고리,거래금액,거래번호,유닉스시간,가맹점_위도,가맹점_경도,사기여부,카테고리_평균비,평균대비_n배,이전거래일시,카테고리_번호,이전거래여부,시간차초
0,2019-01-01 00:00:44,[CARD],"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,[TRANS],1325376044,49.159047,-118.186462,0,126.8,0.8,NaT,1,0,-1.0
1,2019-01-01 00:12:34,[CARD],"fraud_Schultz, Simonis and Little",grocery_pos,44.71,[TRANS],1325376754,40.079588,-74.848087,0,126.8,0.4,NaT,1,0,-1.0
2,2019-01-01 00:17:16,[CARD],fraud_Kling-Grant,grocery_net,46.28,[TRANS],1325377036,40.021888,-74.228188,0,51.4,0.9,NaT,2,0,-1.0
3,2019-01-01 00:20:15,[CARD],fraud_Deckow-O'Conner,grocery_pos,64.09,[TRANS],1325377215,42.360426,-83.552316,0,126.8,0.5,NaT,1,0,-1.0
4,2019-01-01 00:23:41,[CARD],fraud_Balistreri-Nader,misc_pos,25.58,[TRANS],1325377421,29.737426,-90.853194,0,72.4,0.4,NaT,3,0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,[CARD],fraud_Botsford and Sons,home,134.26,[TRANS],1388534208,34.853497,-78.664158,0,55.6,2.4,2020-12-31 23:30:29,11,1,1579.0
491130,2020-12-31 23:56:57,[CARD],"fraud_Baumbach, Hodkiewicz and Walsh",shopping_pos,25.49,[TRANS],1388534217,35.008839,-81.475156,0,82.9,0.3,2020-12-31 22:29:20,10,1,5257.0
491131,2020-12-31 23:59:09,[CARD],fraud_Hoppe-Parisian,kids_pets,111.84,[TRANS],1388534349,29.661049,-96.186633,0,55.0,2.0,2020-12-31 21:16:46,14,1,9743.0
491132,2020-12-31 23:59:15,[CARD],fraud_Rau-Robel,kids_pets,86.88,[TRANS],1388534355,46.658340,-119.715054,0,55.0,1.6,2020-12-31 23:47:11,14,1,724.0
